In [12]:
from google.maps import places_v1
from google.maps.places_v1 import types as places_types
from google.type import latlng_pb2
import pandas as pd
import time
import numpy as np
from dotenv import load_dotenv

In [ ]:
load_dotenv()
client = places_v1.PlacesClient()

In [3]:
# Define the approximate bounding box for Egypt.
lat_min, lat_max = 22.0, 32.0
lng_min, lng_max = 25.0, 34.8

# Define grid step sizes (in degrees).
lat_step = 0.5  # roughly 55 km per 0.5° of latitude.
lng_step = 0.5  # roughly 55 km per 0.5° of longitude (varies by latitude).

# Define search radius in meters.
radius = 50000  # 50 km

In [4]:
# Generate grid centers for the searches.
latitudes = np.arange(lat_min, lat_max, lat_step)
longitudes = np.arange(lng_min, lng_max, lng_step)
search_centers = [(lat, lng) for lat in latitudes for lng in longitudes]
len(latitudes)
len(search_centers)

400

In [5]:

# Define the queries you want to search for.
place_queries = [
    "restaurant",
    "cafe",
    "museum",
    "art gallery",
    "tourist attraction",
    "monument"
]

# Define the fields to retrieve from the Place Details request.
# Essentials IDs Only SKU:
fields_list = [
    "attributions", "id", "name", "photos",
    # Essentials SKU:
    "addressComponents", "adrFormatAddress", "formattedAddress", "location", "plusCode", "shortFormattedAddress", "types", "viewport",
    # Pro SKU:
    "accessibilityOptions", "businessStatus", "containingPlaces", "displayName", "googleMapsLinks", "googleMapsUri", "iconBackgroundColor", "iconMaskBaseUri", "primaryType", "primaryTypeDisplayName", "pureServiceAreaBusiness", "subDestinations", "utcOffsetMinutes",
    # Enterprise SKU:
    "currentOpeningHours", "currentSecondaryOpeningHours", "internationalPhoneNumber", "nationalPhoneNumber", "priceLevel", "priceRange", "rating", "regularOpeningHours", "regularSecondaryOpeningHours", "userRatingCount", "websiteUri"
]
fields_param = ",".join(fields_list)

In [6]:
# Counter to ensure we don't exceed 10,000 details requests.
details_requests_count = 0
max_details_requests = 10000

# List to store detailed place responses.
all_detailed_places = []

In [ ]:
# Loop over grid centers and place types.
for center in search_centers:
    latitude, longitude = center

    # Create a LocationRestriction using the inner class from SearchNearbyRequest.
    location_restriction = places_types.SearchNearbyRequest.LocationRestriction()
    # Set the circle using a LatLng with correct field names.
    location_restriction.circle.center = latlng_pb2.LatLng(latitude=latitude, longitude=longitude)
    location_restriction.circle.radius = float(radius)
    
    for place_type in place_queries:
        print(f"Searching for '{place_type}' near ({latitude}, {longitude})...")
        
        # Build the SearchNearbyRequest using "included_types" instead of "query".
        request = places_types.SearchNearbyRequest(
            location_restriction=location_restriction,
            included_types=[place_type]
            metadata=()
        )
        
        try:
            response = client.search_nearby(request=request)
        except Exception as e:
            print(f"Error during search: {e}")
            continue
        
        if response.places:
            for result in response.places:
                if details_requests_count >= max_details_requests:
                    print("Reached maximum details requests limit (10,000).")
                    break
                
                # Construct resource name in the format "places/{place_id}"
                resource_name = f"places/{result.id}"
                try:
                    details_request = places_types.GetPlaceRequest(
                        name=resource_name,
                        fields=fields_param
                    )
                    details_response = client.get_place(request=details_request)
                    if details_response:
                        all_detailed_places.append(details_response)
                        details_requests_count += 1
                except Exception as e:
                    print(f"Error fetching details for {resource_name}: {e}")
            if details_requests_count >= max_details_requests:
                break
        time.sleep(1)  # pause between queries to avoid rate limits
    if details_requests_count >= max_details_requests:
        break

print(f"Total details requests made: {details_requests_count}")


Searching for 'restaurant' near (22.0, 25.0)...
Error during search: 400 FieldMask is a required parameter. See https://cloud.google.com/apis/docs/system-parameters on how to provide it. As an example, you can set the header 'X-Goog-FieldMask' to value 'places.displayName', 'places.id' to ask for the display name and the place id of a place. You can also set the value to '*' in manual testing to get all the available response fields.

Searching for 'cafe' near (22.0, 25.0)...
Error during search: 400 FieldMask is a required parameter. See https://cloud.google.com/apis/docs/system-parameters on how to provide it. As an example, you can set the header 'X-Goog-FieldMask' to value 'places.displayName', 'places.id' to ask for the display name and the place id of a place. You can also set the value to '*' in manual testing to get all the available response fields.

Searching for 'museum' near (22.0, 25.0)...
Error during search: 400 FieldMask is a required parameter. See https://cloud.google

KeyboardInterrupt: 

In [ ]:
# Deduplicate results by place_id.
unique_places = {}
for place in all_detailed_places:
    unique_places[place.place_id] = place
unique_places_list = list(unique_places.values())

In [ ]:
# Convert the detailed place responses into a pandas DataFrame.
data = []
for place in unique_places_list:
    data.append({
        "place_id": place.place_id,
        "resource_name": place.name,  # contains the place resource name (places/PLACE_ID)
        "display_name": getattr(place, "displayName", None),
        "attributions": place.attributions,
        "photos": place.photos,
        "addressComponents": place.address_components,
        "adrFormatAddress": getattr(place, "adrFormatAddress", None),
        "formattedAddress": getattr(place, "formattedAddress", None),
        "location": place.location,
        "plusCode": getattr(place, "plusCode", None),
        "shortFormattedAddress": getattr(place, "shortFormattedAddress", None),
        "types": place.types,
        "viewport": getattr(place, "viewport", None),
        "accessibilityOptions": getattr(place, "accessibilityOptions", None),
        "businessStatus": getattr(place, "businessStatus", None),
        "containingPlaces": getattr(place, "containingPlaces", None),
        "googleMapsLinks": getattr(place, "googleMapsLinks", None),
        "googleMapsUri": getattr(place, "googleMapsUri", None),
        "iconBackgroundColor": getattr(place, "iconBackgroundColor", None),
        "iconMaskBaseUri": getattr(place, "iconMaskBaseUri", None),
        "primaryType": getattr(place, "primaryType", None),
        "primaryTypeDisplayName": getattr(place, "primaryTypeDisplayName", None),
        "pureServiceAreaBusiness": getattr(place, "pureServiceAreaBusiness", None),
        "subDestinations": getattr(place, "subDestinations", None),
        "utcOffsetMinutes": getattr(place, "utcOffsetMinutes", None),
        "currentOpeningHours": getattr(place, "currentOpeningHours", None),
        "currentSecondaryOpeningHours": getattr(place, "currentSecondaryOpeningHours", None),
        "internationalPhoneNumber": getattr(place, "internationalPhoneNumber", None),
        "nationalPhoneNumber": getattr(place, "nationalPhoneNumber", None),
        "priceLevel": getattr(place, "priceLevel", None),
        "priceRange": getattr(place, "priceRange", None),
        "rating": getattr(place, "rating", None),
        "regularOpeningHours": getattr(place, "regularOpeningHours", None),
        "regularSecondaryOpeningHours": getattr(place, "regularSecondaryOpeningHours", None),
        "userRatingCount": getattr(place, "userRatingCount", None),
        "websiteUri": getattr(place, "websiteUri", None)
    })

df_places = pd.DataFrame(data)
print("Final DataFrame head:")
print(df_places.head())